In [7]:
#Import useful python modules
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

#Import Skit-learn libs
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, MinMaxScaler, Imputer


import os
print(os.listdir("datasets"))


['application_test.csv', 'application_train.csv', 'bureau.csv', 'bureau_balance.csv', 'credit_card_balance.csv', 'HomeCredit_columns_description.csv', 'installments_payments.csv', 'POS_CASH_balance.csv', 'previous_application.csv', 'sample_submission.csv']


In [8]:
#Read train and test sets
df_train = pd.read_csv("datasets/application_train.csv")
df_test = pd.read_csv("datasets/application_test.csv")

In [9]:
#Read extra datasets
bureau = pd.read_csv('datasets/bureau.csv').sort_values(['SK_ID_CURR', 'SK_ID_BUREAU']).reset_index(drop = True)
bureau_balance = pd.read_csv('datasets/bureau_balance.csv').sort_values('SK_ID_BUREAU').reset_index(drop = True)
cash = pd.read_csv('datasets/POS_CASH_balance.csv').sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index(drop = True)
credit = pd.read_csv('datasets/credit_card_balance.csv').sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index(drop = True)
previous = pd.read_csv('datasets/previous_application.csv').sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index(drop = True)
installments = pd.read_csv('datasets/installments_payments.csv').sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index(drop = True)

In [10]:
# set up label variable as y
y = df_train['TARGET']

**Manual Feature Engineering**

In [11]:
# Add a column 'AGE' in the datasets.
df_train['AGE']=df_train['DAYS_BIRTH'] / -365
df_test['AGE']=df_test['DAYS_BIRTH'] / -365 

In [12]:
df_train['AGE'].describe()

count    307511.000000
mean         43.936973
std          11.956133
min          20.517808
25%          34.008219
50%          43.150685
75%          53.923288
max          69.120548
Name: AGE, dtype: float64

In [13]:
#Drop the 'DAYS_BIRTH' column
df_train.drop(['DAYS_BIRTH'],axis=1,inplace=True)
df_test.drop(['DAYS_BIRTH'],axis=1,inplace=True)

In [14]:
#Look at anormaly in 'DAYS_EMPLOYED' column. It has a value 365243.
df_train['DAYS_EMPLOYED'].describe()

count    307511.000000
mean      63815.045904
std      141275.766519
min      -17912.000000
25%       -2760.000000
50%       -1213.000000
75%        -289.000000
max      365243.000000
Name: DAYS_EMPLOYED, dtype: float64

In [15]:
# Replace the anomalous values with 0(not employed)
df_train['DAYS_EMPLOYED'].replace({365243: 0}, inplace = True)
df_test['DAYS_EMPLOYED'].replace({365243: 0}, inplace = True)
df_train['DAYS_EMPLOYED'].plot.hist(title = 'Days Employment Histogram')
plt.xlabel('Days Employment')

Text(0.5,0,'Days Employment')

The following two functions are defined to aggregate useful information  from the extra data sets. Five aggregation functions are used: ['count','max','min','mean','sum'].

In [16]:
def agg_numerical(df,group_var,name):
    #This is a function that aggregates datasets with only numerical columns.
    for col in df.columns:
        if col != group_var and 'SK_ID' in col:
            df.drop(col,axis=1,inplace=True)
    numeric_df = df.select_dtypes('number')
    agg_features = numeric_df.groupby(group_var, as_index=False).agg(['count','max','min','mean','sum']).reset_index()
    feature_name = [group_var]
    for feature in agg_features.columns.levels[0]:
        if 'SK_ID' not in feature:
            for agg in agg_features.columns.levels[1][:-1]:
                feature_name.append(name+'_'+feature+'_'+agg)
    agg_features.columns=feature_name
    return agg_features

In [18]:
def agg_features(df,group_var,name):
    #This is a function that aggregates datasets with both numerical and categorical columns. 
    for col in df.columns:
        if col != group_var and 'SK_ID' in col:
            df.drop(col,axis=1,inplace=True)
    numeric_df = df.select_dtypes('number')
    categoric_df = df.select_dtypes('object')
    categoric_df[group_var] = df[group_var]
    categoric_df = pd.get_dummies(categoric_df)
    agg_numeric = numeric_df.groupby(group_var, as_index=False).agg(['count','max','min','mean','sum']).reset_index()
    agg_categoric = categoric_df.groupby(group_var, as_index=False).agg(['mean','sum']).reset_index()
    
    feature_name1 = [group_var]
    for feature in agg_numeric.columns.levels[0]:
        if 'SK_ID' not in feature:
            for agg in agg_numeric.columns.levels[1][:-1]:
                feature_name1.append(name+'_'+feature+'_'+agg)
    agg_numeric.columns=feature_name1
    
    feature_name2 = [group_var]
    for feature in agg_categoric.columns.levels[0]:
        if 'SK_ID' not in feature:
            for agg in agg_categoric.columns.levels[1][:-1]:
                feature_name2.append(name+'_'+feature+'_'+agg)
    agg_categoric.columns=feature_name2
    
    agg_features = agg_categoric.merge(agg_numeric, on=group_var, how='left')
    
    return agg_features

Aggregate all the datasets

In [19]:
agg_bb = agg_features(bureau_balance,'SK_ID_BUREAU','bureau_balance')
agg_bb.head(10)

C:\Users\yiqichen\UB\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,SK_ID_BUREAU,bureau_balance_STATUS_0_mean,bureau_balance_STATUS_0_sum,bureau_balance_STATUS_1_mean,bureau_balance_STATUS_1_sum,bureau_balance_STATUS_2_mean,bureau_balance_STATUS_2_sum,bureau_balance_STATUS_3_mean,bureau_balance_STATUS_3_sum,bureau_balance_STATUS_4_mean,...,bureau_balance_STATUS_5_sum,bureau_balance_STATUS_C_mean,bureau_balance_STATUS_C_sum,bureau_balance_STATUS_X_mean,bureau_balance_STATUS_X_sum,bureau_balance_MONTHS_BALANCE_count,bureau_balance_MONTHS_BALANCE_max,bureau_balance_MONTHS_BALANCE_min,bureau_balance_MONTHS_BALANCE_mean,bureau_balance_MONTHS_BALANCE_sum
0,5001709,0.000000,0,0.000000,0,0.0,0,0.0,0,0.0,...,0,0.886598,86,0.113402,11,97,0,-96,-48.0,-4656
1,5001710,0.060241,5,0.000000,0,0.0,0,0.0,0,0.0,...,0,0.578313,48,0.361446,30,83,0,-82,-41.0,-3403
2,5001711,0.750000,3,0.000000,0,0.0,0,0.0,0,0.0,...,0,0.000000,0,0.250000,1,4,0,-3,-1.5,-6
3,5001712,0.526316,10,0.000000,0,0.0,0,0.0,0,0.0,...,0,0.473684,9,0.000000,0,19,0,-18,-9.0,-171
4,5001713,0.000000,0,0.000000,0,0.0,0,0.0,0,0.0,...,0,0.000000,0,1.000000,22,22,0,-21,-10.5,-231
5,5001714,0.000000,0,0.000000,0,0.0,0,0.0,0,0.0,...,0,0.000000,0,1.000000,15,15,0,-14,-7.0,-105
6,5001715,0.000000,0,0.000000,0,0.0,0,0.0,0,0.0,...,0,0.000000,0,1.000000,60,60,0,-59,-29.5,-1770
7,5001716,0.313953,27,0.000000,0,0.0,0,0.0,0,0.0,...,0,0.453488,39,0.232558,20,86,0,-85,-42.5,-3655
8,5001717,0.772727,17,0.000000,0,0.0,0,0.0,0,0.0,...,0,0.227273,5,0.000000,0,22,0,-21,-10.5,-231
9,5001718,0.615385,24,0.051282,2,0.0,0,0.0,0,0.0,...,0,0.076923,3,0.256410,10,39,0,-38,-19.0,-741


In [20]:
bureau = bureau.merge(agg_bb, on='SK_ID_BUREAU', how='left')
bureau.head(10)

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,...,bureau_balance_STATUS_5_sum,bureau_balance_STATUS_C_mean,bureau_balance_STATUS_C_sum,bureau_balance_STATUS_X_mean,bureau_balance_STATUS_X_sum,bureau_balance_MONTHS_BALANCE_count,bureau_balance_MONTHS_BALANCE_max,bureau_balance_MONTHS_BALANCE_min,bureau_balance_MONTHS_BALANCE_mean,bureau_balance_MONTHS_BALANCE_sum
0,100001,5896630,Closed,currency 1,-857,0,-492.0,-553.0,NaN,0,...,0.0,0.655172,19.0,0.241379,7.0,29.0,0.0,-28.0,-14.0,-406.0
1,100001,5896631,Closed,currency 1,-909,0,-179.0,-877.0,NaN,0,...,0.0,0.966667,29.0,0.000000,0.0,30.0,0.0,-29.0,-14.5,-435.0
2,100001,5896632,Closed,currency 1,-879,0,-514.0,-544.0,NaN,0,...,0.0,0.620690,18.0,0.310345,9.0,29.0,0.0,-28.0,-14.0,-406.0
3,100001,5896633,Closed,currency 1,-1572,0,-1329.0,-1328.0,NaN,0,...,0.0,0.846154,44.0,0.134615,7.0,52.0,0.0,-51.0,-25.5,-1326.0
4,100001,5896634,Active,currency 1,-559,0,902.0,NaN,NaN,0,...,0.0,0.000000,0.0,0.315789,6.0,19.0,0.0,-18.0,-9.0,-171.0
5,100001,5896635,Active,currency 1,-49,0,1778.0,NaN,NaN,0,...,0.0,0.000000,0.0,0.500000,1.0,2.0,0.0,-1.0,-0.5,-1.0
6,100001,5896636,Active,currency 1,-320,0,411.0,NaN,NaN,0,...,0.0,0.000000,0.0,0.000000,0.0,11.0,0.0,-10.0,-5.0,-55.0
7,100002,6113835,Closed,currency 1,-1043,0,62.0,-967.0,0.0,0,...,0.0,0.125000,2.0,0.187500,3.0,16.0,-19.0,-34.0,-26.5,-424.0
8,100002,6158903,Active,currency 1,-1042,0,780.0,NaN,NaN,0,...,0.0,0.125000,2.0,0.187500,3.0,16.0,-19.0,-34.0,-26.5,-424.0
9,100002,6158904,Closed,currency 1,-1125,0,-1038.0,-1038.0,NaN,0,...,0.0,0.090909,2.0,0.045455,1.0,22.0,-15.0,-36.0,-25.5,-561.0


In [15]:
agg_bureau = agg_features(bureau,'SK_ID_CURR','bureau')
agg_bureau.head(10)

C:\Users\yiqichen\UB\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,SK_ID_CURR,bureau_CREDIT_ACTIVE_Active_mean,bureau_CREDIT_ACTIVE_Active_sum,bureau_CREDIT_ACTIVE_Bad debt_mean,bureau_CREDIT_ACTIVE_Bad debt_sum,bureau_CREDIT_ACTIVE_Closed_mean,bureau_CREDIT_ACTIVE_Closed_sum,bureau_CREDIT_ACTIVE_Sold_mean,bureau_CREDIT_ACTIVE_Sold_sum,bureau_CREDIT_CURRENCY_currency 1_mean,...,bureau_bureau_balance_MONTHS_BALANCE_mean_count,bureau_bureau_balance_MONTHS_BALANCE_mean_max,bureau_bureau_balance_MONTHS_BALANCE_mean_min,bureau_bureau_balance_MONTHS_BALANCE_mean_mean,bureau_bureau_balance_MONTHS_BALANCE_mean_sum,bureau_bureau_balance_MONTHS_BALANCE_sum_count,bureau_bureau_balance_MONTHS_BALANCE_sum_max,bureau_bureau_balance_MONTHS_BALANCE_sum_min,bureau_bureau_balance_MONTHS_BALANCE_sum_mean,bureau_bureau_balance_MONTHS_BALANCE_sum_sum
0,100001,0.428571,3,0.0,0,0.571429,4,0.0,0,1.0,...,7,-0.5,-25.5,-11.785714,-82.5,7,-1.0,-1326.0,-400.000000,-2800.0
1,100002,0.250000,2,0.0,0,0.750000,6,0.0,0,1.0,...,8,-1.5,-39.5,-21.875000,-175.0,8,-6.0,-632.0,-337.625000,-2701.0
2,100003,0.250000,1,0.0,0,0.750000,3,0.0,0,1.0,...,0,NaN,NaN,NaN,0.0,0,NaN,NaN,NaN,0.0
3,100004,0.000000,0,0.0,0,1.000000,2,0.0,0,1.0,...,0,NaN,NaN,NaN,0.0,0,NaN,NaN,NaN,0.0
4,100005,0.666667,2,0.0,0,0.333333,1,0.0,0,1.0,...,3,-1.0,-6.0,-3.000000,-9.0,3,-3.0,-78.0,-30.333333,-91.0
5,100007,0.000000,0,0.0,0,1.000000,1,0.0,0,1.0,...,0,NaN,NaN,NaN,0.0,0,NaN,NaN,NaN,0.0
6,100008,0.333333,1,0.0,0,0.666667,2,0.0,0,1.0,...,0,NaN,NaN,NaN,0.0,0,NaN,NaN,NaN,0.0
7,100009,0.222222,4,0.0,0,0.777778,14,0.0,0,1.0,...,0,NaN,NaN,NaN,0.0,0,NaN,NaN,NaN,0.0
8,100010,0.500000,1,0.0,0,0.500000,1,0.0,0,1.0,...,2,-19.5,-72.5,-46.000000,-92.0,2,-702.0,-2610.0,-1656.000000,-3312.0
9,100011,0.000000,0,0.0,0,1.000000,4,0.0,0,1.0,...,0,NaN,NaN,NaN,0.0,0,NaN,NaN,NaN,0.0


In [16]:
credit.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1489396,100006,-2,0.0,270000,NaN,0.0,NaN,NaN,0.0,...,0.0,0.0,NaN,0,NaN,NaN,0.0,Active,0,0
1,1489396,100006,-1,0.0,270000,NaN,0.0,NaN,NaN,0.0,...,0.0,0.0,NaN,0,NaN,NaN,0.0,Active,0,0
2,1489396,100006,-5,0.0,270000,NaN,0.0,NaN,NaN,0.0,...,0.0,0.0,NaN,0,NaN,NaN,0.0,Active,0,0
3,1489396,100006,-3,0.0,270000,NaN,0.0,NaN,NaN,0.0,...,0.0,0.0,NaN,0,NaN,NaN,0.0,Active,0,0
4,1489396,100006,-4,0.0,270000,NaN,0.0,NaN,NaN,0.0,...,0.0,0.0,NaN,0,NaN,NaN,0.0,Active,0,0


In [17]:
agg_credit = agg_features(credit,'SK_ID_CURR','credit')
agg_credit.head(10)

C:\Users\yiqichen\UB\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,SK_ID_CURR,credit_NAME_CONTRACT_STATUS_Active_mean,credit_NAME_CONTRACT_STATUS_Active_sum,credit_NAME_CONTRACT_STATUS_Approved_mean,credit_NAME_CONTRACT_STATUS_Approved_sum,credit_NAME_CONTRACT_STATUS_Completed_mean,credit_NAME_CONTRACT_STATUS_Completed_sum,credit_NAME_CONTRACT_STATUS_Demand_mean,credit_NAME_CONTRACT_STATUS_Demand_sum,credit_NAME_CONTRACT_STATUS_Refused_mean,...,credit_SK_DPD_count,credit_SK_DPD_max,credit_SK_DPD_min,credit_SK_DPD_mean,credit_SK_DPD_sum,credit_SK_DPD_DEF_count,credit_SK_DPD_DEF_max,credit_SK_DPD_DEF_min,credit_SK_DPD_DEF_mean,credit_SK_DPD_DEF_sum
0,100006,1.000000,6,0.0,0,0.000000,0,0.0,0,0.0,...,6,0,0,0.000000,0,6,0,0,0.000000,0
1,100011,1.000000,74,0.0,0,0.000000,0,0.0,0,0.0,...,74,0,0,0.000000,0,74,0,0,0.000000,0
2,100013,1.000000,96,0.0,0,0.000000,0,0.0,0,0.0,...,96,1,0,0.010417,1,96,1,0,0.010417,1
3,100021,0.411765,7,0.0,0,0.588235,10,0.0,0,0.0,...,17,0,0,0.000000,0,17,0,0,0.000000,0
4,100023,1.000000,8,0.0,0,0.000000,0,0.0,0,0.0,...,8,0,0,0.000000,0,8,0,0,0.000000,0
5,100028,1.000000,49,0.0,0,0.000000,0,0.0,0,0.0,...,49,0,0,0.000000,0,49,0,0,0.000000,0
6,100036,1.000000,12,0.0,0,0.000000,0,0.0,0,0.0,...,12,0,0,0.000000,0,12,0,0,0.000000,0
7,100042,1.000000,84,0.0,0,0.000000,0,0.0,0,0.0,...,84,1,0,0.023810,2,84,0,0,0.000000,0
8,100043,1.000000,33,0.0,0,0.000000,0,0.0,0,0.0,...,33,0,0,0.000000,0,33,0,0,0.000000,0
9,100047,0.972222,35,0.0,0,0.000000,0,0.0,0,0.0,...,36,0,0,0.000000,0,36,0,0,0.000000,0


In [18]:
cash.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1369693,100001,-53,4.0,0.0,Completed,0,0
1,1369693,100001,-54,4.0,1.0,Active,0,0
2,1369693,100001,-57,4.0,4.0,Active,0,0
3,1369693,100001,-55,4.0,2.0,Active,0,0
4,1369693,100001,-56,4.0,3.0,Active,0,0


In [19]:
agg_cash = agg_features(cash,'SK_ID_CURR','cash')
agg_cash.head(10)

C:\Users\yiqichen\UB\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,SK_ID_CURR,cash_NAME_CONTRACT_STATUS_Active_mean,cash_NAME_CONTRACT_STATUS_Active_sum,cash_NAME_CONTRACT_STATUS_Amortized debt_mean,cash_NAME_CONTRACT_STATUS_Amortized debt_sum,cash_NAME_CONTRACT_STATUS_Approved_mean,cash_NAME_CONTRACT_STATUS_Approved_sum,cash_NAME_CONTRACT_STATUS_Canceled_mean,cash_NAME_CONTRACT_STATUS_Canceled_sum,cash_NAME_CONTRACT_STATUS_Completed_mean,...,cash_SK_DPD_count,cash_SK_DPD_max,cash_SK_DPD_min,cash_SK_DPD_mean,cash_SK_DPD_sum,cash_SK_DPD_DEF_count,cash_SK_DPD_DEF_max,cash_SK_DPD_DEF_min,cash_SK_DPD_DEF_mean,cash_SK_DPD_DEF_sum
0,100001,0.777778,7.0,0.0,0,0.0,0,0.0,0,0.222222,...,9,7,0,0.777778,7,9,7,0,0.777778,7
1,100002,1.000000,19.0,0.0,0,0.0,0,0.0,0,0.000000,...,19,0,0,0.000000,0,19,0,0,0.000000,0
2,100003,0.928571,26.0,0.0,0,0.0,0,0.0,0,0.071429,...,28,0,0,0.000000,0,28,0,0,0.000000,0
3,100004,0.750000,3.0,0.0,0,0.0,0,0.0,0,0.250000,...,4,0,0,0.000000,0,4,0,0,0.000000,0
4,100005,0.818182,9.0,0.0,0,0.0,0,0.0,0,0.090909,...,11,0,0,0.000000,0,11,0,0,0.000000,0
5,100006,0.857143,18.0,0.0,0,0.0,0,0.0,0,0.095238,...,21,0,0,0.000000,0,21,0,0,0.000000,0
6,100007,0.939394,62.0,0.0,0,0.0,0,0.0,0,0.045455,...,66,0,0,0.000000,0,66,0,0,0.000000,0
7,100008,0.939759,78.0,0.0,0,0.0,0,0.0,0,0.048193,...,83,1294,0,339.060241,28142,83,0,0,0.000000,0
8,100009,0.890625,57.0,0.0,0,0.0,0,0.0,0,0.109375,...,64,0,0,0.000000,0,64,0,0,0.000000,0
9,100010,0.909091,10.0,0.0,0,0.0,0,0.0,0,0.090909,...,11,0,0,0.000000,0,11,0,0,0.000000,0


In [20]:
installments.head()

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1369693,100001,1.0,1,-1709.0,-1715.0,3951.00,3951.00
1,1369693,100001,1.0,2,-1679.0,-1715.0,3951.00,3951.00
2,1369693,100001,2.0,4,-1619.0,-1628.0,17397.90,17397.90
3,1369693,100001,1.0,3,-1649.0,-1660.0,3951.00,3951.00
4,1851984,100001,1.0,2,-2916.0,-2916.0,3982.05,3982.05


In [21]:
agg_installments = agg_numerical(installments,'SK_ID_CURR','installments')
agg_installments.head(10)

,SK_ID_CURR,installments_NUM_INSTALMENT_VERSION_count,installments_NUM_INSTALMENT_VERSION_max,installments_NUM_INSTALMENT_VERSION_min,installments_NUM_INSTALMENT_VERSION_mean,installments_NUM_INSTALMENT_VERSION_sum,installments_NUM_INSTALMENT_NUMBER_count,installments_NUM_INSTALMENT_NUMBER_max,installments_NUM_INSTALMENT_NUMBER_min,installments_NUM_INSTALMENT_NUMBER_mean,...,installments_AMT_INSTALMENT_count,installments_AMT_INSTALMENT_max,installments_AMT_INSTALMENT_min,installments_AMT_INSTALMENT_mean,installments_AMT_INSTALMENT_sum,installments_AMT_PAYMENT_count,installments_AMT_PAYMENT_max,installments_AMT_PAYMENT_min,installments_AMT_PAYMENT_mean,installments_AMT_PAYMENT_sum
0,100001,7,2.0,1.0,1.142857,8.0,7,4,1,2.714286,...,7,17397.900,3951.000,5885.132143,41195.925,7,17397.900,3951.000,5885.132143,41195.925
1,100002,19,2.0,1.0,1.052632,20.0,19,19,1,10.000000,...,19,53093.745,9251.775,11559.247105,219625.695,19,53093.745,9251.775,11559.247105,219625.695
2,100003,25,2.0,1.0,1.040000,26.0,25,12,1,5.080000,...,25,560835.360,6662.970,64754.586000,1618864.650,25,560835.360,6662.970,64754.586000,1618864.650
3,100004,3,2.0,1.0,1.333333,4.0,3,3,1,2.000000,...,3,10573.965,5357.250,7096.155000,21288.465,3,10573.965,5357.250,7096.155000,21288.465
4,100005,9,2.0,1.0,1.111111,10.0,9,9,1,5.000000,...,9,17656.245,4813.200,6240.205000,56161.845,9,17656.245,4813.200,6240.205000,56161.845
5,100006,16,2.0,1.0,1.125000,18.0,16,10,1,4.437500,...,16,691786.890,2482.920,62947.088438,1007153.415,16,691786.890,2482.920,62947.088438,1007153.415
6,100007,66,2.0,1.0,1.166667,77.0,66,17,1,7.045455,...,66,22678.785,1821.780,12666.444545,835985.340,66,22678.785,0.180,12214.060227,806127.975
7,100008,35,2.0,1.0,1.028571,36.0,35,10,1,5.057143,...,35,432218.295,8003.475,27702.964286,969603.750,35,432218.295,227.160,27360.502714,957617.595
8,100009,51,1.0,1.0,1.000000,51.0,51,12,1,4.549020,...,51,17341.605,6155.280,9568.531765,487995.120,51,17341.605,6155.280,9568.531765,487995.120
9,100010,10,1.0,1.0,1.000000,10.0,10,10,1,5.500000,...,10,27463.410,27321.390,27449.208000,274492.080,10,27463.410,27321.390,27449.208000,274492.080


In [22]:
previous.head()

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,1369693,100001,Consumer loans,3951.000,24835.5,23787.0,2520.0,24835.5,FRIDAY,13,...,Connectivity,8.0,high,POS mobile with interest,365243.0,-1709.0,-1499.0,-1619.0,-1612.0,0.0
1,1038818,100002,Consumer loans,9251.775,179055.0,179055.0,0.0,179055.0,SATURDAY,9,...,Auto technology,24.0,low_normal,POS other with interest,365243.0,-565.0,125.0,-25.0,-17.0,0.0
2,1810518,100003,Cash loans,98356.995,900000.0,1035882.0,NaN,900000.0,FRIDAY,12,...,XNA,12.0,low_normal,Cash X-Sell: low,365243.0,-716.0,-386.0,-536.0,-527.0,1.0
3,2396755,100003,Consumer loans,6737.310,68809.5,68053.5,6885.0,68809.5,SATURDAY,15,...,Consumer electronics,12.0,middle,POS household with interest,365243.0,-2310.0,-1980.0,-1980.0,-1976.0,1.0
4,2636178,100003,Consumer loans,64567.665,337500.0,348637.5,0.0,337500.0,SUNDAY,17,...,Furniture,6.0,middle,POS industry with interest,365243.0,-797.0,-647.0,-647.0,-639.0,0.0


In [23]:
agg_previous = agg_features(credit,'SK_ID_CURR','credit')
agg_previous.head(10)

C:\Users\yiqichen\UB\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,SK_ID_CURR,credit_NAME_CONTRACT_STATUS_Active_mean,credit_NAME_CONTRACT_STATUS_Active_sum,credit_NAME_CONTRACT_STATUS_Approved_mean,credit_NAME_CONTRACT_STATUS_Approved_sum,credit_NAME_CONTRACT_STATUS_Completed_mean,credit_NAME_CONTRACT_STATUS_Completed_sum,credit_NAME_CONTRACT_STATUS_Demand_mean,credit_NAME_CONTRACT_STATUS_Demand_sum,credit_NAME_CONTRACT_STATUS_Refused_mean,...,credit_SK_DPD_count,credit_SK_DPD_max,credit_SK_DPD_min,credit_SK_DPD_mean,credit_SK_DPD_sum,credit_SK_DPD_DEF_count,credit_SK_DPD_DEF_max,credit_SK_DPD_DEF_min,credit_SK_DPD_DEF_mean,credit_SK_DPD_DEF_sum
0,100006,1.000000,6,0.0,0,0.000000,0,0.0,0,0.0,...,6,0,0,0.000000,0,6,0,0,0.000000,0
1,100011,1.000000,74,0.0,0,0.000000,0,0.0,0,0.0,...,74,0,0,0.000000,0,74,0,0,0.000000,0
2,100013,1.000000,96,0.0,0,0.000000,0,0.0,0,0.0,...,96,1,0,0.010417,1,96,1,0,0.010417,1
3,100021,0.411765,7,0.0,0,0.588235,10,0.0,0,0.0,...,17,0,0,0.000000,0,17,0,0,0.000000,0
4,100023,1.000000,8,0.0,0,0.000000,0,0.0,0,0.0,...,8,0,0,0.000000,0,8,0,0,0.000000,0
5,100028,1.000000,49,0.0,0,0.000000,0,0.0,0,0.0,...,49,0,0,0.000000,0,49,0,0,0.000000,0
6,100036,1.000000,12,0.0,0,0.000000,0,0.0,0,0.0,...,12,0,0,0.000000,0,12,0,0,0.000000,0
7,100042,1.000000,84,0.0,0,0.000000,0,0.0,0,0.0,...,84,1,0,0.023810,2,84,0,0,0.000000,0
8,100043,1.000000,33,0.0,0,0.000000,0,0.0,0,0.0,...,33,0,0,0.000000,0,33,0,0,0.000000,0
9,100047,0.972222,35,0.0,0,0.000000,0,0.0,0,0.0,...,36,0,0,0.000000,0,36,0,0,0.000000,0


In [24]:
#Free RAM
bureau,cash,credit,bureau_balance,previous,installments = [],[],[],[],[],[]

Merge the data from extra datasets into the original datasets

In [25]:
all_data = pd.concat((df_train, df_test))
all_data = all_data.merge(agg_bureau, on='SK_ID_CURR', how='left')

C:\Users\yiqichen\UB\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [26]:
all_data = all_data.merge(agg_cash, on='SK_ID_CURR', how='left')
all_data = all_data.merge(agg_credit, on='SK_ID_CURR', how='left')
all_data = all_data.merge(agg_previous, on='SK_ID_CURR', how='left')
all_data = all_data.merge(agg_installments, on='SK_ID_CURR', how='left')
df_train = all_data[:df_train.shape[0]]
df_test = all_data[df_train.shape[0]:]
print('ALL Features shape: ', all_data.shape)
print('Training Features shape: ', df_train.shape)
print('Testing Features shape: ', df_test.shape)

ALL Features shape:  (356255, 634)
Training Features shape:  (307511, 634)
Testing Features shape:  (48744, 634)


In [27]:
#Free RAM
all_data = []
agg_bureau,agg_cash,agg_credit,agg_previous,agg_installments = [],[],[],[],[]

Look into the age distribution of clients

In [ ]:
# Clients age distribution
df_train['AGE'].plot.hist(title = 'AGES',edgecolor = 'k', bins = 25)
plt.title('Age of Client'); plt.xlabel('Age (years)'); plt.ylabel('Count')

In [ ]:
# Clients age distribution with respect to target.
sns.distplot(df_train.loc[df_train['TARGET'] == 0, 'AGE'], label = 'target == 0', hist=False)
sns.distplot(df_train.loc[df_train['TARGET'] == 1, 'AGE'], label = 'target == 1', hist=False)
plt.xlabel('Age (years)'); plt.ylabel('Density'); plt.title('Distribution of Ages'); plt.legend(loc='best')

In [ ]:
age = df_train[['AGE','TARGET']]
age['AGE_BINNED'] = pd.cut(age['AGE'], bins = np.linspace(20, 70, num = 11))
age.head()

In [ ]:
# Devide client age into groups
age_groups = age.groupby('AGE_BINNED').mean().sort_values(by='TARGET')
age_groups

It seems the number of unpaid loans decreases with client age.

In [ ]:
# Convert 'AGE' column into categorical age groups 
combine = [df_train, df_test]
for dataset in combine:
    dataset.loc[(dataset.AGE <= 20.0),'AGE'] = 0
    dataset.loc[(dataset.AGE>20.0) & (dataset.AGE<=25.0),'AGE'] = 1
    dataset.loc[(dataset.AGE>25.0) & (dataset.AGE<=30.0),'AGE'] = 2
    dataset.loc[(dataset.AGE>30.0) & (dataset.AGE<=35.0),'AGE'] = 3
    dataset.loc[(dataset.AGE>35.0) & (dataset.AGE<=40.0),'AGE'] = 4
    dataset.loc[(dataset.AGE>40.0) & (dataset.AGE<=45.0),'AGE'] = 5
    dataset.loc[(dataset.AGE>45.0) & (dataset.AGE<=50.0),'AGE'] = 6
    dataset.loc[(dataset.AGE>50.0) & (dataset.AGE<=55.0),'AGE'] = 7
    dataset.loc[(dataset.AGE>55.0) & (dataset.AGE<=60.0),'AGE'] = 8
    dataset.loc[(dataset.AGE>60.0) & (dataset.AGE<=65.0),'AGE'] = 9
    dataset.loc[(dataset.AGE>65.0) & (dataset.AGE<=70.0),'AGE'] = 10
    dataset.loc[(dataset.AGE>70.0),'AGE'] = 11   

Drop the 'TARGET' column from the datasets and save the 'ID' column.

In [ ]:
df_train.drop(['TARGET'],axis=1,inplace=True)
df_train.drop(['SK_ID_CURR'], axis=1, inplace=True)
test_ID = df_test['SK_ID_CURR']
df_test.drop(['SK_ID_CURR'], axis=1, inplace=True)

In [ ]:
#Onehot encoding: Convert categorical columns into matrices
all_data = pd.concat((df_train, df_test))
all_data = pd.get_dummies(all_data)
df_train = all_data[:df_train.shape[0]]
df_test = all_data[df_train.shape[0]:]
print('ALL Features shape: ', all_data.shape)
print('Training Features shape: ', df_train.shape)
print('Testing Features shape: ', df_test.shape)

In [ ]:
# Free the RAM
all_data=[]

In [ ]:
# Normalization of the training and testing sites.
xtrain = df_train.copy()
xtest = df_test.copy()
scaler = MinMaxScaler(feature_range = (0, 1))
scaler.fit(xtrain)
xtrain = scaler.transform(xtrain)
xtest = scaler.transform(xtest)

In [ ]:
print(xtrain.shape)
print(xtest.shape)

Train an XGBoost model on the dataset.

In [ ]:
# Import fitting methods
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score

In [ ]:
#XGBOOST
import xgboost as xgb

In [ ]:
def xgb_train(data,label,feature_names, num_leaves, max_depth, eta, subsample):
    x_train, x_cv, y_train, y_cv = train_test_split(data, label, test_size=0.2)
    dtrain = xgb.DMatrix(x_train, label=y_train, feature_names=feature_names)
    dcv = xgb.DMatrix(x_cv, label=y_cv, feature_names=feature_names)
    xgb_params = {'eval_metric':'auc','objective':'binary:logistic','eta':eta, 
                  'max_depth':int(max_depth), 'subsample':subsample}
    evallist  = [(dtrain,'train'),(dcv,'eval')]
    xgb_model = xgb.train(xgb_params, dtrain, num_boost_round=int(num_leaves), evals=evallist, early_stopping_rounds=50)
    return xgb_model, dcv, y_cv

In [ ]:
# Use Bayesian Optimization algorithm to find the optimum parameters
from bayes_opt import BayesianOptimization
def optimise_xgb(data, label, feature_names, num_leaves, max_depth, eta, subsample):
    x_train, x_cv, y_train, y_cv = train_test_split(data, label, test_size=0.2)
    dtrain = xgb.DMatrix(x_train, label=y_train, feature_names=feature_names)
    dcv = xgb.DMatrix(x_cv, label=y_cv, feature_names=feature_names)
    def target(num_leaves, max_depth, eta, subsample):
        xgb_params = {'eval_metric':'auc','objective':'binary:logistic','eta':eta, 
                  'max_depth':int(max_depth), 'subsample':subsample}
        evallist  = [(dtrain,'train'),(dcv,'eval')]
        xgbm = xgb.train(xgb_params, dtrain, num_boost_round=int(num_leaves), 
                              evals=evallist, early_stopping_rounds=50, verbose_eval=True)
        return roc_auc_score(y_cv, xgbm.predict(dcv))
    
    bo = BayesianOptimization(target,
                             {'num_leaves':(10,100),
                             'max_depth':(3,100),
                             'eta':(0.001,0.5),
                             'subsample':(0,1)})
    bo.maximize(init_points=2, n_iter=5, acq='ucb', kappa=10)
    return bo

In [ ]:
feature_names = df_train.columns

Call the optimise_xgb function to find the best parameters for this model. 

In [ ]:
#bo = optimise_xgb(xtrain, y, feature_names, num_leaves=(10,100), 
#                      max_depth=(3,50), eta=(0.001,0.5), subsample=(0,1))

In [ ]:
#print(bo.res['max']['max_params'])

After running the function, I manually code in the parameters for the optimum model.

In [ ]:
xgb_model,dcv, y_cv = xgb_train(xtrain, y, feature_names, num_leaves=600, 
                      max_depth=71, eta=0.08712890217415196, subsample=0.8975383551198788)

In [ ]:
#Check at the validation accuracy for this model
y_scores = xgb_model.predict(dcv)
roc_auc = roc_auc_score(y_cv, y_scores)
print(roc_auc)

Make predictions by the model.

In [ ]:
xtest =df_test[imp_features].copy()
xtest = scaler.transform(xtest)

In [ ]:
dtest = xgb.DMatrix(xtest, feature_names=feature_names)
pred_xgb=xgb_model.predict(dtest)

In [ ]:
print(pred_xgb[:10])